# Fuentes

### Link: https://towardsdatascience.com/simple-image-data-augmentation-technics-to-mitigate-overfitting-in-computer-vision-2a6966f51af4
Explicación general sobre técnicas de **data augmentation** orientadas a conjuntos de datos de imágenes.

### Link: https://albumentations.ai/docs/getting_started/image_augmentation/
Página oficial de la biblioteca **Albumentations**, utilizada para crear pipelines con operaciones aleatorias a realizar sobre las imágenes para generar un conjunto de datos más grande.

### Link: https://medium.com/the-artificial-impostor/custom-image-augmentation-with-keras-70595b01aeac
Es interesante observar cómo se puede hacer aplicar data augmentation 'on the fly' sobre cada batch, para evitar que el conjunto de datos crezca demasiado en memoria y que no pueda manejarse.

In [61]:
import numpy as np

In [62]:
import seaborn as sns

In [63]:
import matplotlib.pyplot as plt

In [64]:
import albumentations as A

In [65]:
sns.set(style='darkgrid', context='notebook')

## Cargando las bases de datos

In [66]:
x_train_valid = np.load('../input/cifar100-nn-competition/x_train.npy')
y_train_valid = np.load('../input/cifar100-nn-competition/y_train.npy')
x_test = np.load('../input/cifar100-nn-competition/x_test.npy')

# Preprocesamiento de entradas para ResNet50

In [67]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [68]:
x_train_valid = preprocess_input(x_train_valid)
x_test = preprocess_input(x_test)

# Separando conjuntos para entrenamiento y validación

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=15, stratify=y_train_valid)

# Data Augmentation with Albumentation
Se aplica **data augmentation** utilizando la biblioteca Albumentations para aumentar el tamaño del conjunto de datos para entrenamiento. El conjunto de validación permanece intacto para validar que la técnica tuvo buenos resultados sin contaminar los datos de dicho conjunto.

In [71]:
from tensorflow.keras.utils import Sequence

In [72]:
from albumentations import Compose, ToFloat, HorizontalFlip, VerticalFlip, Rotate, RandomSizedCrop, ShiftScaleRotate, GridDistortion
from albumentations import ElasticTransform, RandomBrightnessContrast

In [73]:
class AugmentedSequence(Sequence):
  """ Dataset generator with data augmentation """

  def __init__(self, x, y, batch_size, augmentation, shuffle=True):
    """ Create an instance of the data augmented generator, which is a 
        dataset generator to provide 'on the fly' data augmentation.
        @param x
        @param y
        @param batch_size
        @param augmentation
        @param shuffle
    """
    # Save internal parameters of the augmented sequence
    self.x = x
    self.y = y
    self.batch_size = batch_size
    self.augmentation = augmentation
    self.shuffle = shuffle

    # Initialization
    self.on_epoch_end()
  
  def __len__(self):
    """ Compute the length of an epoch measured in batches
    """
    return int(np.floor(len(self.x) / float(self.batch_size)))
  
  def __getitem__(self, index):
    """ Return the item from the sequence at the given index
        @param index
    """
    # Generate indexes of the batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

    # Extract the input and output batch from the original dataset
    batch_x = self.x[indexes]
    batch_y = self.y[indexes]
    
    # Return an augmented version of the batch
    return np.array([
      self.augmentation(image=x)['image'] for x in batch_x
    ]), np.array(batch_y)

  def on_epoch_end(self):
    """ Updates indexes after each epoch
    """
    self.indexes = np.arange(len(self.x))
    if self.shuffle is True:
        np.random.shuffle(self.indexes)


In [74]:
# Create the AugmentedSequence
album_generator = AugmentedSequence(x_train,
                                    y_train,
                                    40000,
                                    Compose([
                                        ShiftScaleRotate(shift_limit=0.1,
                                                         scale_limit=0.2,
                                                         rotate_limit=30,
                                                         p=0.5),
                                        HorizontalFlip(p=0.5),
                                        VerticalFlip(p=0.5),
                                        GridDistortion(p=0.2),
                                        ElasticTransform(p=0.2),
                                        RandomBrightnessContrast(p=0.2)
                                        ])
                                    )

# Modelos

In [75]:
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout, Conv2D, MaxPooling2D, InputLayer, AveragePooling2D, Input, UpSampling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow import keras

## ResNet 50

In [76]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [77]:
# It's important to 
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256,256,3))

In [78]:
# Train weigths that contains batch normalization
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False # Originally, this was False!

In [79]:
# Create layers
model = Sequential()

# Performing upsampling to fit better imagenet images dimension
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())

# Loading ResNet Model.
model.add(resnet_model)

# GlobalAVGPooling outputs 1x1?
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization()) # This is new also

# Fully connected network
# Layer 1
model.add(Dense(units=1024))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.4))

# Layer 2
model.add(Dense(units=1024))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.4))

# Softmax layer
model.add(Dense(units=100))
model.add(BatchNormalization())
model.add(Activation('softmax'))


# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [80]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_res50.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
epochs = 5
batch_size = 64
augmented_factor = 11
for i in range(augmented_factor):
    print(f'******** Iteration {i+1} of {augmented_factor+1} ********')
    batch_x, batch_y = album_generator[0]
    model.fit(batch_x,
            batch_y, 
            validation_data=(x_valid, y_valid), 
            callbacks=[mc_callback],
            batch_size=batch_size,
            epochs=epochs
            )

******** Iteration 1 of 12 ********
Epoch 1/5
625/625 [==============================] - 201s 313ms/step - loss: 3.8665 - accuracy: 0.1436 - val_loss: 1.7578 - val_accuracy: 0.5499
Epoch 2/5
625/625 [==============================] - 194s 310ms/step - loss: 2.9109 - accuracy: 0.3167 - val_loss: 1.3775 - val_accuracy: 0.6295
Epoch 3/5
625/625 [==============================] - 194s 310ms/step - loss: 2.6211 - accuracy: 0.3753 - val_loss: 1.2375 - val_accuracy: 0.6564
Epoch 4/5
625/625 [==============================] - 194s 310ms/step - loss: 2.4249 - accuracy: 0.4114 - val_loss: 1.1470 - val_accuracy: 0.6799
Epoch 5/5
625/625 [==============================] - 194s 310ms/step - loss: 2.2622 - accuracy: 0.4421 - val_loss: 1.0939 - val_accuracy: 0.6917
******** Iteration 2 of 12 ********
Epoch 1/5
625/625 [==============================] - 194s 311ms/step - loss: 2.4691 - accuracy: 0.3925 - val_loss: 1.0119 - val_accuracy: 0.7099
Epoch 2/5
625/625 [==============================] - 194s 

In [81]:
# Load the model and show the final metrics
model = keras.models.load_model('model_res50.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.889 Valid: 0.78


In [82]:
def generate_submission(predictions, filepath='submission.csv'):
    """ Generate the .csv file to submit in the challenge
        @param predictions Predictions made by the model from the test dataset
        @param filepath Filepath for the file generated
    """
    df = pd.DataFrame(predictions, columns=['label'])
    df.index.name = 'Id'
    df.to_csv(filepath)

In [83]:
import pandas as pd

In [84]:
# Predict over the test set
y_pred = model.predict(x_test).argmax(axis=-1)

# Save submission
generate_submission(y_pred, filepath='submission1.csv')